In [1]:
from datasets import Dataset
import transformers
import torch
import arabert
import dataset
import emoji

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report as creport
from multiprocessing import Pool


In [4]:
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')
val_df = pd.read_csv('val_dataset.csv')

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)


dataset = {
    'train': train_dataset,
    'val': val_dataset
}

In [5]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('aubmindlab/aragpt2-base')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def preprocess_function(examples):
    return tokenizer(examples['Tweet'], truncation=True, padding='max_length', max_length=84)

tokenized_datasets = {split: dataset[split].map(preprocess_function, batched=True) for split in dataset}


Map:   0%|          | 0/12082 [00:00<?, ? examples/s]

Map:   0%|          | 0/1196 [00:00<?, ? examples/s]

In [6]:
def convert_labels(example):
    example['labels'] = int(example['labels'])  # Ensure labels are integers
    return example

tokenized_datasets = {split: dataset[split].map(convert_labels) for split in dataset}


Map:   0%|          | 0/12082 [00:00<?, ? examples/s]

Map:   0%|          | 0/1196 [00:00<?, ? examples/s]

In [7]:
num_labels = 3

In [8]:
from transformers import GPT2ForSequenceClassification

num_labels = 3 

model = GPT2ForSequenceClassification.from_pretrained('aubmindlab/aragpt2-base', num_labels=num_labels)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at aubmindlab/aragpt2-base and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
import evaluate
from transformers import AutoConfig, GPT2ForSequenceClassification, AutoTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor, InputFeatures


In [10]:
from transformers import GPT2Config

In [11]:
model_name = "aubmindlab/aragpt2-base"
num_labels = 3  
config = GPT2Config.from_pretrained(model_name,num_labels=num_labels, output_attentions=True)
tokenizer = GPT2Tokenizer.from_pretrained(model_name,
                                          do_lower_case=False,
                                          do_basic_tokenize=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def preprocess_function(examples):
    return tokenizer(examples['Tweet'], truncation=True, padding='max_length', max_length=84)

tokenized_datasets = {split: dataset[split].map(preprocess_function, batched=True) for split in dataset}
tokenizer.max_len = 84
model = GPT2ForSequenceClassification.from_pretrained(model_name, config=config)

train_dataset = SingleSentenceClassificationProcessor(mode='classification')
val_dataset = SingleSentenceClassificationProcessor(mode='classification')

train_dataset.add_examples(texts_or_text_and_labels=tokenized_datasets['train']['Tweet'],labels=tokenized_datasets['train']['labels'],overwrite_examples = True)
val_dataset.add_examples(texts_or_text_and_labels=tokenized_datasets['val']['Tweet'],labels=tokenized_datasets['val']['labels'],overwrite_examples = True)
print(train_dataset.examples[0])

train_features = train_dataset.get_features(tokenizer = tokenizer, max_length =84)
val_features = val_dataset.get_features(tokenizer = tokenizer, max_length =84)
print(config)

print(len(train_features))
print(len(val_features))

Map:   0%|          | 0/12082 [00:00<?, ? examples/s]

Map:   0%|          | 0/1196 [00:00<?, ? examples/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at aubmindlab/aragpt2-base and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


InputExample(guid=None, text_a='اليوم يا انا فيني شي يا الناس متسلطه علي ', text_b=None, label=0)
GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "output_attentions": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_param

In [12]:
train_features[5]

InputFeatures(input_ids=[31637, 1566, 3168, 281, 1566, 2045, 38389, 6097, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=None, label=0)

In [13]:
def compute_metrics(p): #p should be of type EvalPrediction
    print(np.shape(p.predictions[0]))
    print(np.shape(p.predictions[1]))
    print(len(p.label_ids))
    preds = np.argmax(p.predictions[0], axis=1)
    assert len(preds) == len(p.label_ids)
    print(classification_report(p.label_ids,preds))
    print(confusion_matrix(p.label_ids,preds))

    macro_f1 = f1_score(p.label_ids,preds,average='macro')
    macro_precision = precision_score(p.label_ids,preds,average='macro')
    macro_recall = recall_score(p.label_ids,preds,average='macro')
    acc = accuracy_score(p.label_ids,preds)
    return {
      'macro_f1' : macro_f1,
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
    }

In [14]:
from transformers import TrainingArguments

In [15]:
training_args = TrainingArguments("./train")
training_args.do_train = True
training_args.evaluate_during_training = True
training_args.lamp_epsilon = 1e-3
training_args.learning_rate = 2e-5
training_args.warmup_steps = 0
training_args.per_device_train_batch_size = 1
training_args.per_device_eval_batch_size = 1
training_args.num_train_epochs = 4
training_args.logging_steps = 500
training_args.save_steps = 500
training_args.seed = 42
print(training_args.logging_steps)

500


In [16]:
import os
import gc
torch.cuda.empty_cache()
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

torch.cuda.empty_cache()

gc.collect()  # Run garbage collection
torch.cuda.empty_cache()  # Clear the GPU cache

torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_max_memory_cached()



C:\Users\Mohamed\anaconda3\Lib\site-packages\torch\cuda\memory.py:329: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(
C:\Users\Mohamed\anaconda3\Lib\site-packages\torch\cuda\memory.py:356: FutureWarning: torch.cuda.reset_max_memory_cached now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


In [17]:
from transformers import Trainer 
trainer = Trainer(model=model,
                  args = training_args,
                  train_dataset =train_features ,
                  eval_dataset = val_features,
                  compute_metrics = compute_metrics,

)


In [18]:
trainer.train()


Step,Training Loss
500,1.534200
1000,1.942700
1500,1.415600
2000,1.399300
2500,1.143300
3000,1.142700
3500,0.938500
4000,1.067400
4500,0.858400
5000,0.874500


TrainOutput(global_step=48328, training_loss=0.4159858638414226, metrics={'train_runtime': 6586.289, 'train_samples_per_second': 7.338, 'train_steps_per_second': 7.338, 'total_flos': 2071791648718848.0, 'train_loss': 0.4159858638414226, 'epoch': 4.0})

In [19]:
model.save_pretrained('./fine-tuned-araGPT_last')
# Save the tokenizer
tokenizer.save_pretrained('./fine-tuned-araGPT_last')

('./fine-tuned-araGPT_last\\tokenizer_config.json',
 './fine-tuned-araGPT_last\\special_tokens_map.json',
 './fine-tuned-araGPT_last\\vocab.json',
 './fine-tuned-araGPT_last\\merges.txt',
 './fine-tuned-araGPT_last\\added_tokens.json')

In [20]:
trainer.evaluate()

(1196, 3)
(12, 1196, 12, 84, 84)
1196
              precision    recall  f1-score   support

           0       0.93      0.94      0.94       496
           1       0.68      0.53      0.59        87
           2       0.98      1.00      0.99       613

    accuracy                           0.94      1196
   macro avg       0.86      0.82      0.84      1196
weighted avg       0.94      0.94      0.94      1196

[[468  22   6]
 [ 33  46   8]
 [  0   0 613]]


{'eval_loss': 0.4390614926815033,
 'eval_macro_f1': 0.8403581712880578,
 'eval_macro_precision': 0.8627579253726342,
 'eval_macro_recall': 0.8240946730935607,
 'eval_accuracy': 0.9423076923076923,
 'eval_runtime': 2144.9031,
 'eval_samples_per_second': 0.558,
 'eval_steps_per_second': 0.558,
 'epoch': 4.0}

In [22]:
max_L  = test_df['Tweet'].apply(lambda x: len(x.split())).max()


In [23]:
from torch.utils.data import TensorDataset, DataLoader
import tqdm

print(torch.cuda.is_available())
device = "cuda" if torch.cuda.is_available() else "cpu"

test_dataset = SingleSentenceClassificationProcessor(mode='classification')

test_dataset.add_examples(texts_or_text_and_labels=test_df['Tweet'],labels=test_df['labels'],overwrite_examples = True)
print(test_dataset.examples[0])

test_features = test_dataset.get_features(tokenizer = tokenizer, max_length =max_L)


input_ids = [i.input_ids for i in test_features]
attention_masks = [i.attention_mask for i in test_features]

# Convert all of our data into torch tensors, the required datatype for our model
inputs = torch.tensor(input_ids)
masks = torch.tensor(attention_masks)

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop,
# with an iterator the entire dataset does not need to be loaded into memory
batch_size = 1
test_data = TensorDataset(inputs, masks)

# do not use shuffle, we need the preds to be in same order
test_dataloader = DataLoader(test_data, batch_size=batch_size)

True
InputExample(guid=None, text_a='في حاجات مينفعش نلفت نظركوا ليها زي الاصول كده يا اتربيتوا عليها يا لا ', text_b=None, label=0)
